In [2]:
import os
import fileinput
import csv

In [3]:
# Find all fourplusinteraction, human .gmt files
filenames = [] # List of all gmt files
for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.endswith("fourplusinteractions_human.gmt") and "with" not in file:
             filenames.append(os.path.join(root, file))

In [4]:
# Read in dataframe (Approved, Alias). For aliases with multiple approved symbols, one approved symbol was selected uar 
hgnc = fileinput.FileInput("/Users/ingridshu/Documents/Icahn_Summer19/KEA3_kinasedata/KEA3webData/hgnc_dictionary.tsv")
mapped = {}
for line in hgnc:
    approved = line.split('\t')[0].strip()
    alias = line.split('\t')[1].strip()
    # there are aliases that map to different approved symbols. Taking the first 
    '''if alias in map:
        print(alias)'''
    mapped[alias] = approved

In [6]:
for fn in filenames:
    fileInput = fileinput.FileInput(fn)
    lib = fn.split('/')[7]
    d = {}
    for line in fileInput:
        if 'Alias' not in line:
            kin = line.split('\t')[0].split('_')[0].upper() # Unmatched kinases
            subs = [''.join(x.split('-')).upper() for x in line.split('\t')[1:]] # Unmatched substrates
            # map kinase name -- trailing "\n"??????
            if kin in mapped:
                kin = mapped[kin].strip()
            else:
                continue
            # map substrate names
            subs = {mapped[x].strip() for x in subs if x in mapped}
            if len(subs) >= 4: # Only keep kinases with four or more interactions
                d[kin] = subs
    fileoutput = open('/Users/ingridshu/Documents/Icahn_Summer19/KEA3_kinasedata/KEA3webData/matched_human-fourplusinteractions/{0}_matched.gmt'.format(fn.split('/')[8]), 'w+')
    for k in d:
        fileoutput.write('{0}_{1}_human\t{2}\n'.format(k, lib, '\t'.join(d[k])))